In [1]:
%load_ext autoreload
%autoreload 2

In [28]:
#import pandas as pd
#from WorkforceSentimentMonitoring.data import get_data

In [23]:
amazon_url = 'http://s3.amazonaws.com/assets.datacamp.com/production/course_935/datasets/500_amzn.csv'
          
amazon = pd.read_csv(amazon_url)
#str(amazon) ### There are both pros and cons in the employee reviews from the data
amazon = amazon[['pros', 'cons']]
print(amazon.shape)
amazon

(500, 2)


,pros,cons
0,You're surrounded by smart people and the proj...,Internal tools proliferation has created a mes...
1,Brand name is great. Have yet to meet somebody...,not the most stimulating work. Good brand name...
2,Good money.Interaction with some great minds i...,No proper growth plan for employees.Difficult ...
3,nice pay and overtime and different shifts,didn't last quite long enough
4,Opportunity for those not afraid to work!,Opportunity takes time up to two plus years!
5,"Pay was good, benefits from day one, meet a lo...",Long hours with only two 15 minute breaks and ...
6,"Got to meet some decent people from jersey, wh...","the hours are too long, you have to stand up t..."
7,The salary is good for the work involved. They...,Management and Human Resources do not communic...
8,"Great company, great people. One of the bigges...","For being such a prestigious company, the bene..."
9,Not too intense. Lots of outdoor activities du...,It was not a very interesting project. The cha...


In [26]:
amazon["review"] = amazon["pros"] + amazon["cons"]
#amazon = amazon['review']
#amazon['review']
#amazon
amazon_reviews = amazon['review']
amazon_reviews

0      You're surrounded by smart people and the proj...
1      Brand name is great. Have yet to meet somebody...
2      Good money.Interaction with some great minds i...
3      nice pay and overtime and different shiftsdidn...
4      Opportunity for those not afraid to work!Oppor...
5      Pay was good, benefits from day one, meet a lo...
6      Got to meet some decent people from jersey, wh...
7      The salary is good for the work involved. They...
8      Great company, great people. One of the bigges...
9      Not too intense. Lots of outdoor activities du...
10     Decent pay, but there are far better opportuni...
11     Co-workers and management teams were generally...
12     good benefits, guaranteed 40hrs a week, paid t...
13     you get a great pay and it helps you out with ...
14     Great pay. Great people. Great.Too hard. Too m...
15     Great chances for promotions. The company is g...
16     Fun working environment, challenging, lots of ...
17     The people are great too

In [ ]:
df = pd.read_csv("/Users/theresasporn/Downloads/pos_neg_df.csv")
df.head()

In [ ]:
from WorkforceSentimentMonitoring.data import get_prepaired_data

In [ ]:
type(df.negatives[0])

In [ ]:
def extract_negative(df):
    """return df with negative reviews and their labels"""
    negatives = df[['negatives']]
    #negatives.loc[:,'sentiment'] = 0 # 0=> negative
    return negatives

In [ ]:
negatives = extract_negative(df)
negatives.head(5)

In [ ]:
from WorkforceSentimentMonitoring.preprocessing import tokenize

In [ ]:
negatives_tokenized = df['negatives'].apply(tokenize)
negatives_tokenized

In [ ]:
def extract_positive(df):
    """return df with positive reviews and their labels"""
    positives = df[['positives']]
    #text.loc[:,'sentiment'] = 1 # 1=> positive
    return positives

In [ ]:
positives = extract_positive(df)
positives.head(5)

In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
from gensim.corpora import Dictionary
import pyLDAvis
import pyLDAvis.gensim

id2word = corpora.Dictionary(negatives_tokenized)
dictionary = id2word
dictionary = dictionary.filter_extremes(no_above=0.75)

texts = negatives_tokenized
corpus = [id2word.doc2bow(text) for text in texts]

ldamallet = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=2, id2word=id2word, iterations=100)

coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()

    
def compute_coherence_values(dictionary, corpus, texts, limit=8, start=2, step=1):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=id2word, alpha=.91)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=4, limit=8, step=2)


max_y = max(coherence_values)  # Find the maximum y value
max_x = coherence_values.index(max(coherence_values))  # Find the x value corresponding to the maximum y value
#xmax = x[numpy.argmax(y)]

optimal_model = model_list[coherence_values.index(max(coherence_values))]

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(optimal_model, corpus, id2word)
vis


In [ ]:
vis = pyLDAvis.gensim.prepare(ldamallet, corpus, id2word)
pyLDAvis.save_html(vis, 'pyLDAvis_negatives_tokenized.html')
#Where 'vis' is the name of your model, like this:


In [ ]:
df = pd.read_csv("/Users/theresasporn/code/theresa-sporn/workforce_sentiment_monitoring/raw_data/train.csv")
df

In [ ]:
df = df[['negatives', 'score_1', "score_2", 'score_3', "score_4", "score_5", "overall"]]
df

In [ ]:
df_pos_work_life_balance = df['negatives'] if df.score_1 > 3
df_pos_work_life_balance 